In [5]:
cd /Users/sandra/work/Bioinformatics/Soybeans

/Users/sandra/work/Bioinformatics/Soybeans


In [1]:
pwd

'/Users/sandra/work/Bioinformatics/Soybeans'

In [8]:
with open('Forrest B100B10 BAC sequence Rhg4.txt', 'r') as forrest_file:
    forrest_seq = forrest_file.read()

forrest_seq[:100]

'> [Glycine max] Forrest B100B10 BAC sequence, Rhg4 \nCCCTCGCGAGTTGGTTCAGCTGCTGCCTGAGGCTGGACGACCTCGCGG'

In [2]:
with open('Forrest B100B10 BAC sequence Rhg4.txt', 'r') as forrest_file:
    forrest_lines = ''
    file_line = forrest_file.readline().rstrip()
    # forrest_lines += file_line

    while file_line:
        forrest_lines += file_line
        file_line = forrest_file.readline().rstrip()

forrest_lines[:300]


'> [Glycine max] Forrest B100B10 BAC sequence, Rhg4CCCTCGCGAGTTGGTTCAGCTGCTGCCTGAGGCTGGACGACCTCGCGGAGTTCTACCGGCAGTGCAAATCCGTCGGCATCCAGGAAACCAGCAGCGGCTATCCGCGCATCCATGCCCCCGAACTGCAGGAGTGGGGAGGCACGATGGCCGCTTTGGTCGACCTGCAGGGGGGGGGGGCGCTGAGGTCTGCCTCGTGAAGAAGGTGTTGCTGACTCATACCAGGCCTGAATCGCCCCATCATCCAGCCAGA'

#### The file containing the Forrest sequence I am working with starts with the FASTA format header. This header doesn't help much in downstreadm analysis, so I need to find index where actual sequence starts.

In [4]:
forrest_lines.find('Rhg4')

46

In [8]:
forrest_lines[46]

'R'

Since Python starts index at 0, Rhg4 (name of the region/locus) is indices 46-49. Actual Forrest sequence starts at index 50.

In [3]:
forrest_lines[46:50]

'Rhg4'

In [7]:
forrest_lines[50:55]

'CCCTC'

#### I am proceeding to searching for presence of genes in the ranges of alignment. I am starting with the ranges of alignment for the soybean cultivar Williams chromosomes 8 and 5, since when Nucleotide BLAST, they gave the highest total score and E value. Using the Forrest Rhg4 sequence, I am adjusting the boundaries given by BLAST for the FASTA format header.

In [4]:
start_codon = 'ATG'
sequence = forrest_lines[1036+50:40811+50]
sequence.find(start_codon)


58

In [9]:
stop_codons = ['TAA', 'TAG', 'TGA']
sequence = forrest_lines[1036+50:40811+50]

stop_codon_indices = []
for codon in stop_codons:
    index = sequence.find(codon)
    stop_codon_indices.append(index)

print(f'indices of stop codons in the 1037-40810 range of alignment: {stop_codon_indices}')

indices of stop codons in the 1037-40810 range of alignment: [16, 6, 59]


In [17]:
# trying to reverse the stop codons

stop_codon_indices = []
for codon in stop_codons:
    index = sequence.find(codon[::-1])
    stop_codon_indices.append(index)

stop_codon = sequence[263:266][::-1]
print(f'stop indices reversed: {stop_codon_indices}')
print(f'stop codon: {stop_codon}')

print(sequence[263:266])

stop indices reversed: [57, 98, 266]
stop codon: CAA
AAC


In [13]:
# loop over the sequence when looking for start codon

start_codon = 'ATG'
start_codon_indices = []
sequence = forrest_lines[1036+50:40811+50]

index = sequence.find(start_codon)
while index >= 0:
    start_codon_indices.append(index)
    index = sequence.find(start_codon, index+len(start_codon))

print(f'number of possible start codons: {len(start_codon_indices)}')
print(f'first 10 indices of start codon: {start_codon_indices[:10]}')


number of possible start codons: 734
first 10 indices of start codon: [58, 146, 211, 306, 381, 522, 689, 1011, 1074, 1090]


In [14]:
# finding TATA boxes
tata_box = 'TATAAA'
sequence = forrest_lines[1036+50:40811+50]
tata_indices = []

index = sequence.find(tata_box)
while index >= 0:
    tata_indices.append(index)
    index = sequence.find(tata_box, index + len(tata_box))

print(f'number of TATA boxes: {len(tata_indices)}')
print(f'first 10 indices of TATA boxes: {tata_indices[:10]}')

number of TATA boxes: 51
first 10 indices of TATA boxes: [204, 376, 432, 1998, 2031, 3615, 3944, 4539, 5088, 5185]


Since TATA boxes are 25-35 base pairs upstream of the transcription start site in most eukaryotic genes, we can rule out codons at indices 58, 146, 211 as start codons and presence of genes. Starting gene search from index 1419 (start of sequence + index of TATA box + length of the TATA box + 25 bp downstream of the TATA box)

In [17]:
# start codons starting from 25 bp downstream of the TATA box
codon = 'ATG'
sequence = forrest_lines[1419:40811]

poss_start_codons = []
index = sequence.find(codon)
while index >= 0:
    poss_start_codons.append(index)
    index = sequence.find(codon, index + len(codon))

print(f'number of possible start codons: {len(poss_start_codons)}')

number of possible start codons: 729


In [18]:
# stop codons starting from 35 bp downstream of the TATA box
stop_codons = ['TAA', 'TAG', 'TGA']
sequence = forrest_lines[1419+10:40811]

poss_stop_codons = []
for codon in stop_codons:
    index = sequence.find(codon)
    while index >= 0:
        poss_stop_codons.append(index)
        index = sequence.find(codon, index + len(codon))

print(f'number of possible stop codons: {len(poss_stop_codons)}')

number of possible stop codons: 2528


We got possibile start and stop codons. Let's calculate the difference between indices for each pair to get a sense of presence of genes.

In [19]:
# iterating through the list of possible start codons
poss_genes = []
for i_start in range(len(poss_start_codons)):
    for i_stop in range(len(poss_stop_codons)):
        length = i_stop - i_start
        poss_genes.append(length)

# expecting to have 729 possible genes (same as number of possible start codons), but check
print(f'number of possible genes: {len(poss_genes)}')

number of possible genes: 1842912


In [20]:
# iterating through the list of possible start codons
poss_genes = []
for i_start in range(len(poss_start_codons)):
    for i_stop in range(len(poss_stop_codons)):
        length = i_stop - i_start
        print(f'stop codon - start codon: {length}')
        poss_genes.append(length)

# expecting to have 729 possible genes (same as number of possible start codons), but check
print(f'number of possible genes: {len(poss_genes)}')

stop codon - start codon: 0
stop codon - start codon: 1
stop codon - start codon: 2
stop codon - start codon: 3
stop codon - start codon: 4
stop codon - start codon: 5
stop codon - start codon: 6
stop codon - start codon: 7
stop codon - start codon: 8
stop codon - start codon: 9
stop codon - start codon: 10
stop codon - start codon: 11
stop codon - start codon: 12
stop codon - start codon: 13
stop codon - start codon: 14
stop codon - start codon: 15
stop codon - start codon: 16
stop codon - start codon: 17
stop codon - start codon: 18
stop codon - start codon: 19
stop codon - start codon: 20
stop codon - start codon: 21
stop codon - start codon: 22
stop codon - start codon: 23
stop codon - start codon: 24
stop codon - start codon: 25
stop codon - start codon: 26
stop codon - start codon: 27
stop codon - start codon: 28
stop codon - start codon: 29
stop codon - start codon: 30
stop codon - start codon: 31
stop codon - start codon: 32
stop codon - start codon: 33
stop codon - start codon

KeyboardInterrupt: 

In [24]:
print(f'possible start codons[:10]: {poss_start_codons[:10]}')
print(f'possible stop codons[:10]: {poss_stop_codons[:10]}')

possible start codons[:10]: [48, 189, 356, 678, 741, 757, 834, 885, 933, 1014]
possible stop codons[:10]: [10, 27, 35, 65, 91, 102, 171, 177, 262, 423]


In [25]:
# iterating through the list of possible start codons
poss_genes = []
for start in poss_start_codons:
    for stop in poss_stop_codons:
        if stop > start:
            length = stop - start
            # print(f'stop codon - start codon: {length}')
            poss_genes.append(length)

# expecting to have 729 possible genes (same as number of possible start codons), but check
print(f'number of possible genes: {len(poss_genes)}')

number of possible genes: 899469
